<a href="https://colab.research.google.com/github/hwangsaeyeon/DepressionChatbot/blob/main/kogpt2%EB%AC%B8%EC%9E%A5%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modules Install

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

#my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
#os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
#sys.path.insert(0, my_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-c15j03xu
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-c15j03xu
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit cb17fe90ec3b04b680e1b425431e3201529501c5
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebo

##Import

In [ ]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=128):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = '<usr>'
        self.a_token = '<sys>'
        self.sent_token = '<unused1>'
        self.eos = '</s>'
        self.mask = '<unused0>'
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]

        q = turn["hs"]  # 질문을 가져온다.
        #q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["ss"]  # 답변을 가져온다.
        #a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        sent = turn['label']

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token + str(sent)) #<usr> + 질문 + <unused1> + 감정
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos) #<sys> + 답변 + </s>
        a_len = len(a_toked)



        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #----3. label_ids----#
        # 답변 생성
        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]

        # 질문의 길이만큼 mask + 답변
        labels = [self.mask,] * q_len + a_toked[1:]
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        #----2. mask----#
        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        #----1. token_ids----#
        # token_ids = + 질문문장 + + 감정 + + 답변 + + pad_token_id
        # 질문 + 답변을 index 로 만든다.
        save_input = q_toked + a_toked
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


In [ ]:
#tokenizer
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token="</s>", eos_token="</s>", unk_token='<unk>',
            pad_token="<pad>", mask_token="<unused0>")

#model
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
#model_path = '/content/drive/MyDrive/우울증챗봇/only_hidoc.pt'
#model.load_state_dict(torch.load(model_path))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

device(type='cuda')

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(np.array(data)).to(device), torch.LongTensor(np.array(mask)).to(device), torch.LongTensor(np.array(label)).to(device)

In [ ]:
train_save = pd.read_csv('/content/train_save.csv',index_col=0)
test_save = pd.read_csv('/content/test_save.csv',index_col=0)
hidoc = pd.read_csv('/content/faq_data.csv',index_col=0)
hidoc2 = pd.read_csv('/content/qna_final.csv',index_col=0)
train_save

,hs,ss,label
0,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.더 좋은 회사가 기다리고 있을지...,천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?좋은 일자리를 ...,0
1,요즘 직장생활이 너무 편하고 좋은 것 같아!우리 회사는 복지가 정말 좋아! 내가 곧...,직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?예비...,0
2,취업해야 할 나이인데 취업하고 싶지가 않아.아직 조금 더 놀고 싶은 마음이 커. 인...,취업하고 싶지 않으시군요. 혹시 어떤 이유가 있을까요?아직은 인생을 즐기고 싶은 마...,0
3,우리 회사는 정말 사내 분위기가 좋아.즐거운 일이 매일 생길 거 같은 기분이야. 내...,사내 분위기가 좋아서 즐거우시군요.원하는 진로에 맞춰서 취업해서 더욱 만족스럽군요.,0
4,오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.회사에서 내가 제안한 프...,직장이 좋으시군요. 자세히 알려주실 수 있나요?존중하며 소통하는 직장에 크게 만족감...,0
...,...,...,...
51713,아이가 어렸을 때 맞벌이하느라 제대로 돌봐주지 못했습니다. 늘 의기소침해 있고 짜증...,청소년의 비행은 반사회적 인격장애의 전조증상인 경우도 있고 우울증 때문일 수도 많습...,2
51714,우울증으로 약물치료를 받은 지 한달 정도 되었어요. 의사 선생님은 저더러 좋아졌다고...,아닙니다. 항우울제로 치료할 경우 우울증의 여러 가지 증상들 중에 먼저 좋아지는 것...,2
51715,뇌졸중 후 우울증에 걸린 것 같습니다. 어떻게 해야 할까요?,우울증의 증상으로는 가장 흔하게 식욕 감소와 불면증이 있습니다. 이 밖에 피로감을 ...,2
51716,현재 우울증때문에 항우울제를 먹고 있는데요. 이거 앞으로 복용 중단 못하게 되나요?...,"정신과 약을 장기간 사용시 의존성이 생각다고 생각을 하는 경우가 많은데, 항우울제의...",2


In [ ]:
hidoc['label'] = 2
hidoc = hidoc.drop(['title'],axis=1)
hidoc.columns = ['hs','ss','label']

hidoc2['label'] = 2
hidoc2 = hidoc2.drop(['title'],axis=1)
hidoc2.columns = ['hs','ss','label']
#2116
hidoc_final = pd.concat([hidoc, hidoc2])
hidoc_final = hidoc_final.reset_index(drop=True)
hidoc_train = hidoc_final[:1900]
hidoc_test = hidoc_final[1900:]

In [ ]:
5700/2

In [ ]:
one = train_save[train_save.label==1].sample(n=2850,replace=False)
zero = train_save[train_save.label==0].sample(n=2850,replace=False)
data = pd.concat([zero,one])
#train = pd.concat([data, hidoc_train])
train = train.reset_index(drop=True)
train.to_csv('/content/drive/MyDrive/우울증챗봇/train.csv')

In [ ]:
324

In [ ]:
one = test_save[test_save.label==1].sample(n=324,replace=False)
zero = test_save[test_save.label==0].sample(n=216,replace=False)
data = pd.concat([zero,one])
test = pd.concat([data, hidoc_test])
test = test.reset_index(drop=True)
test.to_csv('/content/drive/MyDrive/우울증챗봇/test.csv')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/우울증챗봇/train.csv',index_col=0)
test = pd.read_csv('/content/drive/MyDrive/우울증챗봇/test.csv',index_col=0)

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
len(train)

5698

In [ ]:
train_set = ChatbotDataset(train, max_len=512)
test_set = ChatbotDataset(test, max_len=512)
train_dataloader = DataLoader(train_set, batch_size=4, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
learning_rate = 5e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 15
Sneg = -1e18

In [ ]:
from tqdm import tqdm
loss_history = []
#token_history = []
#out_history = []
print ("start")

for ep in tqdm(range(epoch), position=0, leave=True):
    for batch_idx, samples in tqdm(enumerate(train_dataloader),position=0,leave=True):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        #token_history.append(token_ids)
        #out_history.append(out)
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
        #pbar.update()
    print()
    print('avg_loss => ', avg_loss.item())
    loss_history.append(avg_loss.item())
print ("end")

start


1425it [15:10,  1.57it/s]
  7%|▋         | 1/15 [15:10<3:32:29, 910.67s/it]


avg_loss =>  107.37537384033203


1425it [15:15,  1.56it/s]
 13%|█▎        | 2/15 [30:26<3:17:54, 913.46s/it]


avg_loss =>  106.12056732177734


...

 93%|█████████▎| 14/15 [3:33:34<15:15, 915.63s/it]

avg_loss =>  86.64906311035156


1425it [15:14,  1.56it/s]


100%|██████████| 15/15 [3:48:49<00:00, 915.30s/it]

avg_loss =>  25.163841247558594
end


In [ ]:
!nvidia-smi

Wed Jun 14 00:54:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |   1671MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

11946342912
13277069312


In [ ]:
for batch_idx, samples in enumerate(train_dataloader):
    del samples
    torch.cuda.empty_cache()
    print(torch.cuda.memory_allocated())
    print(torch.cuda.memory_reserved())
    break

14928666112
15103688704


In [ ]:
import gc
del out
torch.cuda.empty_cache()
gc.collect()

298

In [ ]:
torch.save(model.state_dict(), '/content/kogpt2_0618.pt')  # 모델 객체의 state_dict 저장

In [ ]:
import torch
file_path = ('/content/drive/MyDrive/우울증챗봇/kogpt2_0618.pt')
model.load_state_dict(torch.load(file_path))
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
eval

,hs,label
0,펀드에 넣어둔 자금이 많이 올랐어.응 아주 신이 나. 생각보다 짧은 기간에 반등했어.,0
1,십 년 사귄 애인과 나는 연락이 잘 안 돼도 서로 믿고 있어.우린 서로를 믿고 있고...,0
2,일찍 자고 일찍 일어났더니 공부할 때 마음이 한결 편안해.생활 패턴이 올바르니까 능...,0
3,음식이 맛있다는 게 새삼 감사한 일이야. 우리 아내는 솜씨가 좋거든.밖에서도 아내 ...,0
4,이번에 내가 로또에 당첨됐어.이제 우리 가정에도 경제적인 여유가 생기겠어.,0
...,...,...
642,인터넷에서 우울증이며 불면증에 효과가있다는 아이도스 mp3 파일을 받았습니다..근래...,2
643,우울증? 불안증?3달전에 여자친구와 헤어졌습니다.1년정도 사귀다가 헤어지게 된 것인...,2
644,중국에서 사업을 2년간 하다 첫사업 실패로 인해 몸과 마음이 큰 상처를 받았습니다....,2
645,27세 여성입니다..4달가까이 불면증 인것같아요.. 우울증도 있는건지..4달전에 너...,2


In [ ]:
eval = pd.read_csv('./data_eval (1).csv',index_col=0)
eval = eval.dropna()

for i in range(len(eval['label'])):
    if eval['label'][i] == 2:
        test['label'][i] = 1
    else:
        test['label'][i] = eval['label'][i]

<ipython-input-35-a381248f9b0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][i] = eval['label'][i]
<ipython-input-35-a381248f9b0f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][i] = 1


In [ ]:
test

,hs,ss,label
0,펀드에 넣어둔 자금이 많이 올랐어.응 아주 신이 나. 생각보다 짧은 기간에 반등했어.,이익이 많이 생겨서 기분이 좋으시군요.펀드 이익이 많이 올라서 좋으시겠어요.,0
1,십 년 사귄 애인과 나는 연락이 잘 안 돼도 서로 믿고 있어.우린 서로를 믿고 있고...,정말 좋을 거 같아요. 서로에 대한 믿음이 있네요.깨지지 않게 하기 위해선 어떻게 ...,0
2,일찍 자고 일찍 일어났더니 공부할 때 마음이 한결 편안해.생활 패턴이 올바르니까 능...,편안해서 좋으셨겠어요. 일찍 자고 일찍 일어나는 게 어떤 부분에서 도움이 됐나요?성...,0
3,음식이 맛있다는 게 새삼 감사한 일이야. 우리 아내는 솜씨가 좋거든.밖에서도 아내 ...,아내 분의 음식 솜씨가 좋으셔서 감사하시겠어요.아내 분에게 감사함을 전해드리려면 뭘...,0
4,이번에 내가 로또에 당첨됐어.이제 우리 가정에도 경제적인 여유가 생기겠어.,로또에 당첨되셨군요.정말 다행이세요.,0
...,...,...,...
642,인터넷에서 우울증이며 불면증에 효과가있다는 아이도스 mp3 파일을 받았습니다..근래...,"치료라는 것은, 엄격하게 법률상으로 다스리고 있습니다.그래서, 아무나 치료를 못하게...",1
643,우울증? 불안증?3달전에 여자친구와 헤어졌습니다.1년정도 사귀다가 헤어지게 된 것인...,애인으로 사귀다가 친구로 만나기는 어렵습니다.<o:p> </o:p>헤어지신 게 잘한...,1
644,중국에서 사업을 2년간 하다 첫사업 실패로 인해 몸과 마음이 큰 상처를 받았습니다....,중국에서 받으신 심리적 타격으로 인해불안증 및 우울증이 발생한 것이 의심됩니다.가...,1
645,27세 여성입니다..4달가까이 불면증 인것같아요.. 우울증도 있는건지..4달전에 너...,"1) 먼저, 약물이나, 다른 치료를 사용하지 않는 스스로 지켜야 하는 몇 가지를 소...",1


In [ ]:
len(test['hs'][433])

409

In [ ]:
import torch
model.eval()
# 답변 생성 루프
answer_dataset = []
with torch.no_grad():
    for i in range(len(test)):
        # 질문 및 문맥 설정
        q = test['hs'][i]
        if len(q)>128:
            q=test['hs'][i][:128]
        sent = test['label'][i]
        a = ""
        # 입력 시퀀스 생성
        input_ids = koGPT2_TOKENIZER.encode("<usr>" + q + "<unused1>" + str(sent) + "<sys>" + a, return_tensors="pt").to(device)

        # 모델에 입력 전달하여 답변 생성
        outputs = model.generate(input_ids,no_repeat_ngram_size=3, max_length=128, num_beams=3, do_sample=False, pad_token_id=koGPT2_TOKENIZER.pad_token_id)

        # 생성된 답변 디코딩
        generated_answer = koGPT2_TOKENIZER.decode([el.item() for el in outputs[0]], skip_special_tokens=True)
        gen = generated_answer

        # 생성된 답변 출력
        print(i)
        #print("Chatbot : ", gen)
        answer_dataset.append(gen)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
pd.DataFrame(answer_dataset).to_csv('test_data_generated_비교.csv')

In [ ]:
import torch
# 답변 생성 루프
answer_dataset = []
dataset = ['안녕하세요','저 우울증인가요?','너무 기쁘다']
sentiment = [0,1,0]
with torch.no_grad():
    for i in range(len(dataset)):
w
        q = dataset[i]
        sent = sentiment[i]
        a = ""
        # 입력 시퀀스 생성
        input_ids = koGPT2_TOKENIZER.encode("<usr>" + q + "<unused1>" + str(sent) + "<sys>" + a, return_tensors="pt").to(device)

        # 모델에 입력 전달하여 답변 생성
        outputs = model.generate(input_ids, max_length=128, num_beams=5, pad_token_id=koGPT2_TOKENIZER.pad_token_id)

        # 생성된 답변 디코딩
        generated_answer = koGPT2_TOKENIZER.decode(outputs[0], skip_special_tokens=True)
        gen = generated_answer

        # 생성된 답변 출력
        print("Chatbot : ", gen)
        answer_dataset.append(gen)

Chatbot :  우리는 누구나 여러 가지 스트레스 때문에 우울하게 되는 것을 경험하면서 삽니다. 하지만 그 정도가 지나쳐서 일상생활에 지장을 초래할 정도가 되면 우울증이라고 부릅니다. 우울증은 심하더라도 치료하면 대개 좋아질 수가 있습니다. 노인분들은 만성질환이 많고 외로우며 경제적으로 어려운 경우가 많으므로 젊은 사람보다 더 우울증이 많습니다. 물론 이런 신체적, 사회적 악조건을 갖고 있다고 해서 누구나 우울증이 오는 것은 아닙니다. 우울증이 있는 노인들이 호소하는 특징 중의 하나는 자기 증상을 애기할때 ‘기분이 우울하다’라고 말하기 보다는 다른 신체 증상, 예를 들면 ‘
Chatbot :  우울증은 우울기분, 흥미나 즐거움의 상실, 식욕감소와 체중감소, 불면, 초조, 피로와 활력 상실, 죄책감, 사고력이나 집중력의 감소, 죽음에 대한 반복적인 생각 등의 증상이 나타납니다. 그러므로 꼭 우울한 기분이 없어도 청소년에서는 기분이 과민한 것을 보일 수도 있고, 다른 증상들을 통해 우울증으로 판단할 수 있습니다.
Chatbot :  우리는 누구나 여러 가지 스트레스 때문에 우울하게 되는 것을 경험하면서 삽니다. 하지만 그 정도가 지나쳐서 일상생활에 지장을 초래할 정도가 되면 우울증이라고 부릅니다. 우울증은 심하더라도 치료하면 대개 좋아질 수가 있습니다. 노인분들은 만성질환이 많고 외로우며 경제적으로 어려운 경우가 많으므로 젊은 사람보다 더 우울증이 많습니다. 물론 이런 신체적, 사회적 악조건을 갖고 있다고 해서 누구나 우울증이 오는 것은 아닙니다. 우울증이 있는 노인들이 호소하는 특징 중의 하나는 자기 증상을 애기할때 ‘기분이 우울하다’라고 말하기 보다는 다른 신체 증상, 예를 들면 ‘


In [ ]:
pd.DataFrame(answer_dataset).to_csv('hidoc_generated_3.csv')